# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [42]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [43]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [44]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [45]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [46]:
# 1. 10 countries with the lowest infant mortality rates

import pandas as pd
import numpy as np

# read the xml file
document = ET.parse( './data/mondial_database.xml' )

# create empty lists to store country names and infant mortaility rates
country_list = []
infant_mortality_list = []

# fill out lists for country names and infant mortaility rates
# where infant mortaility rate is missing, fill out NA
for element in document.iterfind('country'):
    country_list.append(element.find('name').text)
    try: 
        infant_mortality_list.append(element.find('infant_mortality').text)
    except:
        infant_mortality_list.append(np.nan)
        
# convert lists to dataframe and get the 10 countries with the lowest infant mortality rates
df = pd.DataFrame({'country':country_list,'infant_mortality_rate':infant_mortality_list}).dropna()
df.sort_values('infant_mortality_rate').head(10)

,country,infant_mortality_rate
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


In [47]:
# 2. 10 cities with the largest population

# create empty lists to store city names and population
city_list = []
population_list = []

# fill out lists for country names and infant mortaility rates
# where there are multiple population values for each city, use the latese one
# where population is missing, fill out NA
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        city_list.append(subelement.find('name').text)
        all_population = [pop.text for pop in subelement.findall('population')]
        if all_population:
            population_list.append(all_population[-1])
        else:
            population_list.append(np.nan)
            
# convert lists to dataframe
df = pd.DataFrame({'city':city_list,'population':population_list}).dropna()

# change data type of population to float
df['population'] = df['population'].astype(int)

# get the 10 cities with the largest population
df.sort_values('population',ascending=False).head(10)

,city,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [48]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

# create empty lists to store names, base population and percentage for ethnic groups
eg_name_list = []
eg_base_list = []
eg_pct_list = []

# for each country, get the folling data from the xml:
# the latest population estimate
# name of each ethnic group
# percentage of each ethnic group
for element in document.iterfind('country'):
    eg = [eg.text for eg in element.findall('ethnicgroup')]
    pct = [eg.attrib['percentage'] for eg in element.findall('ethnicgroup')]
    pop = [pop.text for pop in element.findall('population')]
    if len(eg) * len(pop) == 0:
        continue
    eg_name_list.extend(eg)
    eg_pct_list.extend(pct)
    eg_base_list.extend([pop[-1] for i in range(len(eg))])

# create a dataframe from the lists and calculate population for each ethnic group
df = pd.DataFrame({'ethnic_group':eg_name_list,'base':eg_base_list,'pct':eg_pct_list})
df['population'] = (df['base'].astype(float) * df['pct'].astype(float) / 100).astype(int)

# remove unnecessary columns
df = df.drop(['base','pct'],1)

# get the 10 ethnic groups with the largest overall populations
df.groupby('ethnic_group').sum()['population'].sort_values(ascending=False).head(10)

ethnic_group
Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
Name: population, dtype: int32

In [49]:
#4a. name and country of longest river

# create a dictionary to store the relationship bwtween country code and country name
car_code_dict = {}
for child in document.getroot():
    try:
        car_code_dict[child.attrib['car_code']] = child.find('name').text
    except:
        continue

# create lists to store river names, river lengths and country codes of rivers
r_name_list = []
r_len_list = []
r_ccode_list = []
for element in document.iterfind('river'):
    try:
        r_len_list.append(float(element.find('length').text))
        r_name_list.append(element.find('name').text)
        r_ccode_list.append(element.attrib['country'])
    except:
        continue

# convert lists to a dataframe and find the name and country of longest river
df = pd.DataFrame({'river_name':r_name_list,'river_length':r_len_list,'river_ccode':r_ccode_list})
top1 = df.sort_values('river_length',ascending=False).iloc[0]
name = top1.river_name
country = [car_code_dict[code] for code in top1.river_ccode.split()]

print ('The name of the longest river is {0}. It is in {1}'.format(name,country))

The name of the longest river is Amazonas. It is in ['Colombia', 'Brazil', 'Peru']


In [50]:
# 4b) name and country of largest lake

# create lists to store lake names, lake areas and country codes of lakes
l_name_list = []
l_area_list = []
l_ccode_list = []
for element in document.iterfind('lake'):
    try:
        l_area_list.append(float(element.find('area').text))
        l_name_list.append(element.find('name').text)
        l_ccode_list.append(element.attrib['country'])
    except:
        continue

# convert lists to a dataframe and find the name and country of largest lake
df = pd.DataFrame({'lake_name':l_name_list,'lake_area':l_area_list,'lake_ccode':l_ccode_list})
top1 = df.sort_values('lake_area',ascending=False).iloc[0]
name = top1.lake_name
country = [car_code_dict[code] for code in top1.lake_ccode.split()]

print ('The name of the largest river is {0}. It is in {1}'.format(name,country))


The name of the largest river is Caspian Sea. It is in ['Russia', 'Azerbaijan', 'Kazakhstan', 'Iran', 'Turkmenistan']


In [51]:
# 4c) name and country of airport at highest elevation

# create lists to store airports names, airport elevations and country codes of airports
a_name_list = []
a_elevation_list = []
a_ccode_list = []
for element in document.iterfind('airport'):
    try:
        a_elevation_list.append(float(element.find('elevation').text))
        a_name_list.append(element.find('name').text)
        a_ccode_list.append(element.attrib['country'])
    except:
        continue

# convert lists to a dataframe and find the name and country of airport at highest elevation
df = pd.DataFrame({'airport_name':a_name_list,'airport_elevation':a_elevation_list,'airport_ccode':a_ccode_list})
top1 = df.sort_values('airport_elevation',ascending=False).iloc[0]
name = top1.airport_name
country = [car_code_dict[code] for code in top1.airport_ccode.split()]

print ('The name of the airport at highest elevation is {0}. It is in {1}'.format(name,country))


The name of the airport at highest elevation is El Alto Intl. It is in ['Bolivia']
